In [4]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [5]:
adeft_grounder = AdeftGrounder()

In [6]:
shortforms = ['IR']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [7]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [8]:
list(zip(longforms, counts))

[('ionizing radiation', 3657),
 ('insulin resistance', 2614),
 ('insulin receptor', 1937),
 ('ischemia reperfusion', 1551),
 ('irradiation', 591),
 ('immunoreactivity', 363),
 ('infrared', 328),
 ('ionising radiation', 194),
 ('ischemia and reperfusion', 118),
 ('inverted repeat', 90),
 ('immediate release', 87),
 ('ischaemia reperfusion', 78),
 ('intron retention', 53),
 ('injury', 39),
 ('iterative reconstruction', 38),
 ('ischemic reperfusion', 35),
 ('inversion recovery', 31),
 ('intergenic region', 23),
 ('incidence rate', 23),
 ('immune response', 22),
 ('internal rotation', 21),
 ('infrared spectroscopy', 19),
 ('infrared radiation', 19),
 ('isolated rearing', 17),
 ('interventional radiology', 16),
 ('infection rate', 15),
 ('it receptor', 14),
 ('immune reconstitution', 13),
 ('input resistance', 12),
 ('induced resistance', 12),
 ('intermediate risk', 11),
 ('infusion reaction', 11),
 ('implantation rate', 11),
 ('ice recrystallization', 10),
 ('infra red', 10),
 ('imidazolin

In [9]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [10]:
names

{'MESH:D011839': 'Radiation, Ionizing',
 'MESH:D007333': 'Insulin Resistance',
 'HGNC:6091': 'INSR',
 'MESH:D015427': 'Reperfusion Injury',
 'MESH:D011827': 'Radiation',
 'MESH:D015424': 'Reperfusion',
 'MESH:D014947': 'Wounds and Injuries',
 'MESH:D012399': 'Rotation',
 'MESH:D011871': 'Radiology',
 'GO:GO:0038023': 'signaling receptor activity',
 'MESH:D000074243': 'Immune Reconstitution',
 'MESH:D012306': 'Risk',
 'HGNC:3094': 'DYRK3',
 'CHEBI:CHEBI:74061': 'Ile-Arg',
 'MESH:D012106': 'Research',
 'MESH:D011995': 'Recombination, Genetic',
 'GO:GO:0048511': 'rhythmic process',
 'CHEBI:CHEBI:24433': 'group',
 'CHEBI:CHEBI:29016': 'arginine',
 'MESH:D012220': 'Rhinitis',
 'MESH:D020481': 'Index',
 'DOID:DOID:12510': 'retinal ischemia',
 'MESH:D012160': 'Retina',
 'MESH:D011878': 'Radiotherapy',
 'HGNC:9958': 'REN',
 'EFO:0000989': 'root structure',
 'MESH:D007395': 'Interneurons',
 'MESH:D008027': 'Light',
 'EFO:0001216': 'MDAMB468',
 'MESH:D007259': 'Infrared Rays'}

In [11]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-01 03:58:33] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [12]:
result = [grounding_map, names, pos_labels]

In [13]:
result

[{'group ii': 'ungrounded',
  'ice recrystallization': 'ungrounded',
  'identification rate': 'ungrounded',
  'idiopathic rhinitis': 'MESH:D012220',
  'ile arg': 'CHEBI:CHEBI:74061',
  'illegitimate recombination': 'ungrounded',
  'imaging reconstruction': 'ungrounded',
  'imatinib resistance': 'ungrounded',
  'imidazoline receptor': 'ungrounded',
  'immediate release': 'ungrounded',
  'immune reactive': 'ungrounded',
  'immune reconstitution': 'ungrounded',
  'immune response': 'ungrounded',
  'immunoreactivity': 'ungrounded',
  'implantation rate': 'ungrounded',
  'implementation research': 'ungrounded',
  'in receptor': 'ungrounded',
  'inactive renin': 'HGNC:9958',
  'inadequate response': 'ungrounded',
  'incidence rate': 'ungrounded',
  'incomplete response': 'ungrounded',
  'incomplete revascularization': 'ungrounded',
  'indirect revascularization': 'ungrounded',
  'induced resistance': 'ungrounded',
  'induction ratio': 'ungrounded',
  'infection rate': 'ungrounded',
  'infect

In [26]:
grounding_map, names, pos_labels = [{'group ii': 'ungrounded',
  'ice recrystallization': 'ungrounded',
  'identification rate': 'ungrounded',
  'idiopathic rhinitis': 'MESH:D012220',
  'ile arg': 'CHEBI:CHEBI:74061',
  'illegitimate recombination': 'ungrounded',
  'imaging reconstruction': 'ungrounded',
  'imatinib resistance': 'ungrounded',
  'imidazoline receptor': 'ungrounded',
  'immediate release': 'ungrounded',
  'immune reactive': 'ungrounded',
  'immune reconstitution': 'ungrounded',
  'immune response': 'ungrounded',
  'immunoreactivity': 'ungrounded',
  'implantation rate': 'ungrounded',
  'implementation research': 'ungrounded',
  'in receptor': 'ungrounded',
  'inactive renin': 'HGNC:9958',
  'inadequate response': 'ungrounded',
  'incidence rate': 'ungrounded',
  'incomplete response': 'ungrounded',
  'incomplete revascularization': 'ungrounded',
  'indirect revascularization': 'ungrounded',
  'induced resistance': 'ungrounded',
  'induction ratio': 'ungrounded',
  'infection rate': 'ungrounded',
  'infection response': 'ungrounded',
  'inferior rectus': 'ungrounded',
  'inflammatory response': 'ungrounded',
  'infra red': 'MESH:D007259',
  'infrared': 'MESH:D007259',
  'infrared light': 'MESH:D007259',
  'infrared radiation': 'MESH:D007259',
  'infrared spectroscopy': 'ungrounded',
  'infrarenal': 'ungrounded',
  'infusion reaction': 'ungrounded',
  'inhibition rate': 'ungrounded',
  'inhibitory receptor': 'ungrounded',
  'initial rate': 'ungrounded',
  'initial region': 'ungrounded',
  'initial respond': 'ungrounded',
  'injury': 'MESH:D014947',
  'inner retina': 'MESH:D012160',
  'input resistance': 'ungrounded',
  'insulin receptor': 'HGNC:6091',
  'insulin receptor antibody': 'ungrounded',
  'insulin release': 'ungrounded',
  'insulin resistance': 'MESH:D007333',
  'inter repeat': 'ungrounded',
  'intergenic region': 'ungrounded',
  'intermediate': 'ungrounded',
  'intermediate range': 'ungrounded',
  'intermediate risk': 'ungrounded',
  'intermediate root': 'EFO:0000989',
  'internal': 'ungrounded',
  'internal rotation': 'ungrounded',
  'interneurons': 'MESH:D007395',
  'intervening region': 'ungrounded',
  'interventional radiology': 'ungrounded',
  'intrarectal': 'ungrounded',
  'intrinsic rhythm': 'ungrounded',
  'intron retention': 'ungrounded',
  'inversion recovery': 'ungrounded',
  'inverted repeat': 'ungrounded',
  'inwardly rectification': 'ungrounded',
  'inwardly rectifying': 'ungrounded',
  'inwardly rectifying potassium current': 'ungrounded',
  'ionic radiation': 'MESH:D011839',
  'ionising radiation': 'MESH:D011839',
  'ionization radiation': 'MESH:D011839',
  'ionizing radiation': 'MESH:D011839',
  'ionizing radiotherapy': 'MESH:D011839',
  'ionotropic receptor': 'ungrounded',
  'ir': 'HGNC:6091',
  'ir alone': 'ungrounded',
  'ir group': 'ungrounded',
  'ir treated': 'ungrounded',
  'iressa resistance mda mb 468': 'EFO:0001216',
  'irradiation': 'MESH:D011839',
  'irradiation with 4 gy': 'MESH:D011839',
  'ischaemia reperfusion': 'MESH:D015427',
  'ischemia and reperfusion': 'MESH:D015427',
  'ischemia following by reperfusion': 'MESH:D015427',
  'ischemia reperfusion': 'MESH:D015427',
  'ischemia reperfusion model': 'MESH:D015427',
  'ischemiareperfusion': 'MESH:D015427',
  'ischemic reperfusion': 'MESH:D015427',
  'isolated rearing': 'ungrounded',
  'isoleucine arginine': 'ungrounded',
  'isotonic regression': 'ungrounded',
  'it receptor': 'ungrounded',
  'iterative reconstruction': 'ungrounded',
  'reperfusion injury group': 'ungrounded',
  'resistance index': 'ungrounded',
  'retinal ischemia': 'DOID:DOID:12510'},
 {'MESH:D012220': 'Rhinitis',
  'CHEBI:CHEBI:74061': 'Ile-Arg',
  'HGNC:9958': 'REN',
  'MESH:D007259': 'Infrared Rays',
  'MESH:D014947': 'Wounds and Injuries',
  'MESH:D012160': 'Retina',
  'HGNC:6091': 'INSR',
  'MESH:D007333': 'Insulin Resistance',
  'EFO:0000989': 'root structure',
  'MESH:D007395': 'Interneurons',
  'MESH:D011839': 'Radiation, Ionizing',
  'EFO:0001216': 'MDAMB468',
  'MESH:D015427': 'Reperfusion Injury',
  'DOID:DOID:12510': 'retinal ischemia'},
 ['HGNC:6091', 'MESH:D011839']]

In [27]:
excluded_longforms = ['ir', 'ir alone', 'ir group', 'ir treated']

In [28]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [29]:
additional_entities = {}

In [30]:
unambiguous_agent_texts = {}

In [31]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [32]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-01 04:19:16] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-01 04:24:30] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9635676642180975 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [33]:
classifier.stats

{'label_distribution': {'ungrounded': 784,
  'HGNC:6091': 1512,
  'MESH:D007259': 304,
  'MESH:D011839': 3296,
  'HGNC:9958': 3,
  'MESH:D015427': 1193,
  'MESH:D007333': 1894,
  'MESH:D014947': 34,
  'MESH:D012160': 2,
  'MESH:D012220': 4,
  'EFO:0000989': 1,
  'CHEBI:CHEBI:74061': 5,
  'MESH:D007395': 1,
  'EFO:0001216': 1,
  'DOID:DOID:12510': 1},
 'f1': {'mean': 0.963568, 'std': 0.002946},
 'precision': {'mean': 0.961146, 'std': 0.001891},
 'recall': {'mean': 0.966103, 'std': 0.007573},
 'EFO:0000989': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'CHEBI:CHEBI:74061': {'f1': {'mean': 0.2, 'std': 0.4},
  'pr': {'mean': 0.2, 'std': 0.4},
  'rc': {'mean': 0.2, 'std': 0.4}},
 'MESH:D007395': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.853478, 'std': 0.011479},
  'pr': {'mean': 0.854573, 'std': 0.020447},
  'rc': {'mean': 0.852944, 'st

In [37]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [38]:
disamb.dump(model_name, results_path)

In [39]:
print(disamb.info())

Disambiguation model for IR

Produces the disambiguations:
	INSR*	HGNC:6091
	Ile-Arg	CHEBI:CHEBI:74061
	Infrared Rays	MESH:D007259
	Insulin Resistance	MESH:D007333
	Interneurons	MESH:D007395
	MDAMB468	EFO:0001216
	REN	HGNC:9958
	Radiation, Ionizing*	MESH:D011839
	Reperfusion Injury	MESH:D015427
	Retina	MESH:D012160
	Rhinitis	MESH:D012220
	Wounds and Injuries	MESH:D014947
	retinal ischemia	DOID:DOID:12510
	root structure	EFO:0000989

Class level metrics:
--------------------
Grounding          	Count	F1     
Radiation, Ionizing*	3296	0.98309
 Insulin Resistance	1894	0.95072
               INSR*	1512	0.92101
 Reperfusion Injury	1193	0.94422
         Ungrounded	 784	0.85348
      Infrared Rays	 304	0.87597
Wounds and Injuries	  34	    0.0
            Ile-Arg	   5	    0.2
           Rhinitis	   4	    0.6
                REN	   3	    0.2
             Retina	   2	    0.0
     root structure	   1	    0.0
       Interneurons	   1	    0.0
           MDAMB468	   1	    0.0
   retinal ischemia	   

In [40]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1